In [13]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

from ray.tune.registry import register_env
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from pprint import pprint

In [14]:
class CustomEnv(gym.Env):
    def __init__(self, config=None):
        super(CustomEnv, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Discrete(2)  # Example: 2 discrete actions
        self.observation_space = spaces.Box(low=0, high=1, shape=(
            4,), dtype=np.float32)  # Example: 4 continuous states

    def reset(self):
        # Reset the state of the environment to an initial state
        self.state = np.random.rand(4)
        return self.state

    def step(self, action):
        # Apply action and return new state, reward, done, and info
        self.state = np.random.rand(4)
        reward = 1 if action == 1 else 0  # Example reward logic
        done = np.random.rand() > 0.95  # Example termination condition
        return self.state, reward, done, {}

    def render(self, mode="human"):
        pass  # Optional: Implement rendering logic

In [15]:
def env_creator(env_config):
    return CustomEnv(env_config)


register_env("custom_env", env_creator)

In [16]:
# Initialize Ray
ray.init(ignore_reinit_error=True)

# Configure PPO
config = (
    PPOConfig()
    # Use your registered environment
    .environment(env="custom_env", env_config={})
    .framework("torch")  # Use PyTorch or TensorFlow
    .env_runners(num_cpus_per_env_runner=1)
    .training(train_batch_size=4000, gamma=0.99)  # Training parameters
)

# Build the PPO algorithm
algo = config.build()

# Train the model
for i in range(1):
    result = algo.train()
    pprint(result)

2025-05-03 00:16:46,915	INFO worker.py:1718 -- Calling ray.init() again after it has already been called.


2025-05-03 00:16:51,053	WARNING util.py:61 -- Install gputil for GPU system monitoring.
2025-05-03 00:17:51,064	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2025-05-03 00:17:52,820	ERROR actor_manager.py:873 -- Ray error (The actor 6cfe101d367a767713840a7c01000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
2025-05-03 00:17:52,821	ERROR actor_manager.py:873 -- Ray error (The actor 231123bbe50925679d50769501000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 2 out of service.
2025-05-03 00:17:52,822	ERROR 

{'config': {'_disable_action_flattening': False,
            '_disable_execution_plan_api': -1,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_dont_auto_sync_env_runner_states': False,
            '_enable_rl_module_api': -1,
            '_env_to_module_connector': None,
            '_fake_gpus': False,
            '_is_atari': None,
            '_learner_class': None,
            '_learner_connector': None,
            '_model_config': {},
            '_module_to_env_connector': None,
            '_per_module_overrides': {},
            '_prior_exploration_config': {'type': 'StochasticSampling'},
            '_rl_module_spec': None,
            '_tf_policy_handles_more_than_one_loss': False,
            '_torch_grad_scaler_class': None,
            '_torch_lr_scheduler_classes': None,
            '_train_batch_size_per_learner': None,
            '_use_msgpack_checkpoints': False,
            '_validate_con

In [17]:
# # Save the trained model
# checkpoint = algo.save_to_path("")
# print(f"Checkpoint saved at {checkpoint}")

# # Shutdown Ray
# ray.shutdown()

In [18]:
# Initialize Ray
# ray.init(ignore_reinit_error=True)

# # Load the trained model
# config = PPOConfig().environment(env="CartPole-v1", env_config={}).framework("torch")
# algo = config.build_algo()
# algo.restore()  # Path to the saved checkpoint

# Evaluate the model
env = gym.make("CartPole-v1")
state = env.reset()
done = False
total_reward = 0

while not done:
    action = algo.compute_single_action(state)
    state, reward, done, _ = env.step(action)
    total_reward += reward

print(f"Total reward during evaluation: {total_reward}")

# Shutdown Ray
ray.shutdown()

AttributeError: 'SingleAgentEnvRunner' object has no attribute 'get_policy'